In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# Load the stocks data
data = pd.read_excel('israel_analysis.xlsx',index_col=0)
data.head()

,תאריך,מניה,סוג המלצה,בית השקעות,אנליסט,מחיר בעת המלצה,מחיר יעד,מרחק מהיעד,מחיר אחרון
0,19/09/2021,אנרגיאן,קניה,בנק לאומי,אלה פריד,3384.0,4100.0,21.16%,3473.0
1,19/09/2021,אייאיאס,קניה,ווליו בייס,שי ליפמן,26260.0,38600.0,46.99%,27220.0
2,14/09/2021,פריים אנרגי,קניה,פרוסט אנד סאליבן,טיראן רוטמן,1439.0,1930.0,34.12%,1421.0
3,13/09/2021,אלטשולר שחם גמל,קניה,לאומי פרטנרס,מרב פישר שרוני,1829.0,2300.0,25.75%,1858.0
4,12/09/2021,אאורה,קניה,ווליו בייס,שי ליפמן,349.9,600.0,71.48%,394.2


In [3]:
eng_cols = ['date','stock_name','recommendation_type','inv_house',
            'analyst','rec_date_value','dest_value','value_diff','last_price']
data.columns = eng_cols
data.head()

,date,stock_name,recommendation_type,inv_house,analyst,rec_date_value,dest_value,value_diff,last_price
0,19/09/2021,אנרגיאן,קניה,בנק לאומי,אלה פריד,3384.0,4100.0,21.16%,3473.0
1,19/09/2021,אייאיאס,קניה,ווליו בייס,שי ליפמן,26260.0,38600.0,46.99%,27220.0
2,14/09/2021,פריים אנרגי,קניה,פרוסט אנד סאליבן,טיראן רוטמן,1439.0,1930.0,34.12%,1421.0
3,13/09/2021,אלטשולר שחם גמל,קניה,לאומי פרטנרס,מרב פישר שרוני,1829.0,2300.0,25.75%,1858.0
4,12/09/2021,אאורה,קניה,ווליו בייס,שי ליפמן,349.9,600.0,71.48%,394.2


## translate stock name to english

In [4]:
# Download english stock names from 
# https://info.tase.co.il/Heb/MarketData/SecuritiesInfo/Pages/SecuritiesInfo.aspx

In [5]:
names = pd.read_csv('Data_20211015.tsv', sep='\t',error_bad_lines=False,skiprows=2)
names.head()

,Security Number,ISIN,Name(Hebrew),Name(English),Symbol(Hebrew),Symbol(English)
0,373019,IL0003730194,אאורה,AURA,אאורה,AURA
1,3730413,IL0037304131,אאורה אגח י,AURA B10,אארה.ק10,AURA.B10
2,3730504,IL0037305047,אאורה אגח טו,AURA B 15,אארה.ק15,AURA.B15
3,3730579,IL0037305799,אאורה אגח טז,AURA B16,אארה.ק16,AURA.B16
4,3730454,IL0037304545,אאורה אגח יב,AURA B12,אארה.ק12,AURA.B12


In [6]:
names = pd.read_csv('Data_20211015.tsv', sep='\t',error_bad_lines=False,skiprows=2)
names = names[['Name(Hebrew)', 'Name(English)','Symbol(English)']]
# remove ' for the join
names['Name(Hebrew)'] = names['Name(Hebrew)'].str.replace('\'','')
# remove multiple spaces
names['Name(Hebrew)'] = names['Name(Hebrew)'].str.replace(' +', ' ',regex=True)
names.columns = ['heb_name','eng_name','eng_sym']
#replace .L to -L as in yahoo finance 
names['eng_sym'] = names['eng_sym'].str.replace('.L','-L',regex=False)
names.head()

,heb_name,eng_name,eng_sym
0,אאורה,AURA,AURA
1,אאורה אגח י,AURA B10,AURA.B10
2,אאורה אגח טו,AURA B 15,AURA.B15
3,אאורה אגח טז,AURA B16,AURA.B16
4,אאורה אגח יב,AURA B12,AURA.B12


In [7]:
merged = data.merge(names,left_on='stock_name',right_on='heb_name',how='left',indicator=True)
print(merged[merged['_merge']=='left_only']['stock_name'].unique())
#all english

['Teva Pharmaceutical' 'Nice Systems' 'Ceragon Networks' 'Elbit Systems'
 'Ormat Technologies' 'Audiocodes' 'Gazit Globe' 'Radware' 'MasterCard'
 'Redhill Biopharma' 'Tower Semiconductor' 'Ituran Location And Control'
 'Athersys' 'Check Point Software Tech' 'Facebook']


In [8]:
#merged['eng_name'] = np.where(merged['_merge']=='left_only',merged['stock_name'],merged['eng_name'])
#merged[merged['_merge']=='left_only'].head(2)

In [9]:
merged = merged[merged['_merge']=='both']

In [10]:
cols = ['date', 'eng_name','recommendation_type', 'inv_house', 'analyst',
       'rec_date_value', 'dest_value', 'value_diff', 'last_price','eng_sym']
merged = merged[cols]
merged.head()

,date,eng_name,recommendation_type,inv_house,analyst,rec_date_value,dest_value,value_diff,last_price,eng_sym
0,19/09/2021,ENERGEAN,קניה,בנק לאומי,אלה פריד,3384.0,4100.0,21.16%,3473.0,ENOG
1,19/09/2021,IES,קניה,ווליו בייס,שי ליפמן,26260.0,38600.0,46.99%,27220.0,IES
2,14/09/2021,PRIME ENERGY,קניה,פרוסט אנד סאליבן,טיראן רוטמן,1439.0,1930.0,34.12%,1421.0,PRIM
3,13/09/2021,ALTSHU FUND PEN,קניה,לאומי פרטנרס,מרב פישר שרוני,1829.0,2300.0,25.75%,1858.0,ASPF
4,12/09/2021,AURA,קניה,ווליו בייס,שי ליפמן,349.9,600.0,71.48%,394.2,AURA


In [11]:
merged['date'].head(500).tail()

495    10/12/2019
496    10/12/2019
497    04/12/2019
498    02/12/2019
499    01/12/2019
Name: date, dtype: object

In [12]:
# take only relevant reccomendation
merged = merged.head(500)
merged['date'] = pd.to_datetime(merged['date'])

### add historical prices

In [13]:
#! pip install yfinance
import yfinance as yf
import concurrent.futures

In [14]:
def get_stock_history(tup):
    
    end_date = tup[0]
    stock_sym = tup[1]
    start_date = end_date -  pd.Timedelta(weeks=26)
    try:
        stocknm = yf.Ticker(stock_sym+'.TA')
        hist = stocknm.history(period="6mo",interval='1mo',start = start_date , end = end_date)
        ret = hist[['Close']].sort_index(ascending=False).T
        ret.rename(index={'Close':stock_sym},inplace=True)
        return ret
    except:
        print(stock_sym)
        return pd.DataFrame(index=[stock_sym],columns=['error'],data=0)

In [15]:
%%time
date_symbol_tuple = list(zip(merged.date,merged.eng_sym))

with concurrent.futures.ThreadPoolExecutor(max_workers=min(int(len(date_symbol_tuple)/13),20)) as executor:
    data = list(executor.map(get_stock_history, date_symbol_tuple))    
    all_data_mt = pd.concat(data)

- LAHAV.TA: No data found for this date range, symbol may be delisted
- DELT.TA: No data found for this date range, symbol may be delisted
- DEDR-L.TA: No data found for this date range, symbol may be delisted
- LAHAV.TA: No data found for this date range, symbol may be delisted
- LAHAV.TA: No data found for this date range, symbol may be delisted
- WILC.TA: No data found for this date range, symbol may be delisted
- DEDR-L.TA: No data found for this date range, symbol may be delisted
- ROBO.TA: No data found for this date range, symbol may be delisted
- DEDR-L.TA: No data found for this date range, symbol may be delisted
- DELT.TA: No data found for this date range, symbol may be delisted
- SONO.TA: No data found for this date range, symbol may be delisted
- ILX.TA: No data found for this date range, symbol may be delisted
- DEDR-L.TA: No data found for this date range, symbol may be delisted
- DELT.TA: No data found for this date range, symbol may be delisted
- BVC.TA: No data found 

In [16]:
all_data_mt = all_data_mt.reset_index()

In [17]:
all_data_mt.tail(2)

Date,index,2018-08-01 00:00:00,2018-09-01 00:00:00,2018-10-01 00:00:00,2018-11-01 00:00:00,2018-12-01 00:00:00,2018-12-02 00:00:00,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,...,2021-08-19 00:00:00,2021-08-24 00:00:00,2021-08-25 00:00:00,2021-08-26 00:00:00,2021-08-30 00:00:00,2021-09-01 00:00:00,2021-09-02 00:00:00,2021-09-05 00:00:00,2021-10-01 00:00:00,2021-10-14 00:00:00
498,DUNI,NaN,6829.00000,63.150002,61.380001,57.34000,NaN,57.900002,66.75,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,LAHAV,155.699997,1.41027,1.412690,1.453080,1.34727,NaN,1.412690,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
merged.tail(2)

,date,eng_name,recommendation_type,inv_house,analyst,rec_date_value,dest_value,value_diff,last_price,eng_sym
498,2019-02-12,DUNIEC,קניה,ווליו בייס,שי ליפמן,11180.0,18700.0,67.26%,15590.0,DUNI
499,2019-01-12,LAHAV,קניה,ווליו בייס,שי ליפמן,213.0,315.0,47.89%,568.0,LAHAV


In [19]:
df = pd.concat([merged, all_data_mt], axis=1)

In [20]:
df.head()

,date,eng_name,recommendation_type,inv_house,analyst,rec_date_value,dest_value,value_diff,last_price,eng_sym,...,2021-08-19 00:00:00,2021-08-24 00:00:00,2021-08-25 00:00:00,2021-08-26 00:00:00,2021-08-30 00:00:00,2021-09-01 00:00:00,2021-09-02 00:00:00,2021-09-05 00:00:00,2021-10-01 00:00:00,2021-10-14 00:00:00
0,2021-09-19,ENERGEAN,קניה,בנק לאומי,אלה פריד,3384.0,4100.0,21.16%,3473.0,ENOG,...,NaN,NaN,NaN,NaN,NaN,3786.000000,NaN,NaN,NaN,NaN
1,2021-09-19,IES,קניה,ווליו בייס,שי ליפמן,26260.0,38600.0,46.99%,27220.0,IES,...,NaN,NaN,NaN,NaN,NaN,27220.000000,NaN,NaN,NaN,NaN
2,2021-09-14,PRIME ENERGY,קניה,פרוסט אנד סאליבן,טיראן רוטמן,1439.0,1930.0,34.12%,1421.0,PRIM,...,NaN,NaN,NaN,NaN,NaN,1356.000000,NaN,NaN,NaN,NaN
3,2021-09-13,ALTSHU FUND PEN,קניה,לאומי פרטנרס,מרב פישר שרוני,1829.0,2300.0,25.75%,1858.0,ASPF,...,NaN,NaN,NaN,NaN,NaN,1768.000000,NaN,NaN,NaN,NaN
4,2021-12-09,AURA,קניה,ווליו בייס,שי ליפמן,349.9,600.0,71.48%,394.2,AURA,...,NaN,NaN,NaN,NaN,NaN,380.799988,NaN,NaN,400.899994,400.899994


In [21]:
df.to_excel("df.xlsx")